#### 자연어 데이터셋 전처리


In [1]:
### 패키지 설치
# %pip install Korpora


In [2]:
### 패키지 설치 확인
import Korpora

Korpora.__version__


'0.2.0'

-   [1] 데이터 준비 <hr>


In [3]:
### 데이터 다운로드
from Korpora import Korpora

# 데이터셋 파일명
data_file = "korean_hate_speech"

# 다운로드 fetch
corpus = Korpora.fetch(data_file)


[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_1.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_2.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_3.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_4.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\unlabeled\unlabeled_comments_5.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\news_title\unlabeled_comments.news_title_1.txt
[Korpora] Corpus `korean hate speech` is already installed at C:\Users\KDP-25\Korpora\korean_hate_speech\news_title\unlabeled_comments.news_title_2.

In [4]:
# 데이터 로딩
corpus = Korpora.load(data_file)



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [5]:
### 데이터 확인
hateDS = corpus.train
hate_valDS = corpus.dev
hate_testDS = corpus.test
unlabelDS = corpus.unlabeled

### 1개 데이터 정보 추출
print(hateDS[0].text)
print(hateDS[0].title)
print(hateDS[0].gender_bias)
print(hateDS[0].bias)
print(hateDS[0].hate)

print(unlabelDS[0])


(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....
"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"
False
others
hate
SentencePair(text='"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"', pair='"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"')


-   [2] 데이터 전처리 <hr>
    -   주제에 따른 데이터 피쳐 & 라벨 선택 ===> 뉴스댓글 + 차별종류
    -   형태소 분석기 결정


In [6]:
### 피쳐 데이터 추출
textData = corpus.get_all_texts()


In [7]:
### 데이터 파일 저장
save_filename = "../data/hate_text.txt"
with open(save_filename, "w", encoding="utf-8") as f:
    for text in textData:
        f.write(text + "\n")


In [10]:
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(
    f"--input={save_filename} --model_prefix=text_bpe --vocab_size=8000 model_type=bpe"
)


In [12]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("text_bpe.model")

sentence = "뭐가 어떻게 굴러가는지 모르겠다..."
sentences = ["일단 책에서 하라는 대로 하고는 있는데", "이렇게 하는게 맞는건가?"]

tokenized_sentence = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = tokenizer.encode_as_pieces(sentences)
print("단일 문장 토큰화 :", tokenized_sentence)
print("여러 문장 토큰화 :", tokenized_sentences)

encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = tokenizer.encode_as_ids(sentences)
print("단일 문장 정수 인코딩 :", encoded_sentence)
print("여러 문장 정수 인코딩 :", encoded_sentences)

decode_ids = tokenizer.decode_ids(encoded_sentences)
decode_pieces = tokenizer.decode_pieces(tokenized_sentences)
print("정수 인코딩에서 문장 변환 :", decode_ids)
print("하위 단어 토큰에서 문장 변환 :", decode_pieces)


단일 문장 토큰화 : ['▁뭐', '가', '▁어떻게', '▁굴', '러', '가는', '지', '▁모르', '겠다', '...']
여러 문장 토큰화 : [['▁일', '단', '▁', '책', '에서', '▁하', '라는', '▁대', '로', '▁하고', '는', '▁있', '는데'], ['▁이렇게', '▁하는', '게', '▁맞', '는', '건', '가', '?']]
단일 문장 정수 인코딩 : [1090, 6711, 2681, 4512, 6897, 2212, 6690, 5174, 335, 7]
여러 문장 정수 인코딩 : [[101, 6738, 6677, 7160, 220, 66, 4300, 44, 6709, 2038, 6736, 146, 773], [1646, 1899, 6806, 309, 6736, 6859, 6711, 6787]]
정수 인코딩에서 문장 변환 : ['일단 책에서 하라는 대로 하고는 있는데', '이렇게 하는게 맞는건가?']
하위 단어 토큰에서 문장 변환 : ['일단 책에서 하라는 대로 하고는 있는데', '이렇게 하는게 맞는건가?']


In [14]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("text_bpe.model")

vocab = {idx: tokenizer.id_to_piece(idx) for idx in range(tokenizer.get_piece_size())}
print(list(vocab.items())[:10])
print("vocab size:", len(vocab))


[(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '""'), (4, '..'), (5, '▁"'), (6, '▁""'), (7, '...'), (8, '종합'), (9, "▁'")]
vocab size: 8000
